In [1]:
import nest_asyncio
import tensorflow as tf
import tensorflow_federated as tff

from ocddetection.learning.federated.simulation import training
from ocddetection.learning.federated.impl import mixed

In [2]:
window_size = 150
batch_size = 128
hidden_size = 64

In [3]:
nest_asyncio.apply()

In [4]:
train, val, _ = training.__load_data(
    '/opportunity/augmented/including_original',
    1,
    window_size,
    batch_size
)

In [5]:
checkpoint_manager = tff.simulation.FileCheckpointManager('/ocd-detection/checkpoints/federated/hybrid')

In [7]:
server_state, client_state_fn, model_fn = mixed.create(
    window_size,
    hidden_size,
    training.__server_optimizer_fn,
    training.__training_metrics_fn
)

In [8]:
ckpt = checkpoint_manager.load_latest_checkpoint([server_state, [client_state_fn(-1) for _ in val.clients]])[0]

In [14]:
weights = ckpt[0].model

In [15]:
client_states = dict(zip(val.clients, ckpt[1]))